<a href="https://colab.research.google.com/github/thisaraniNJ/Aircraft_Damage_Prediction/blob/main/Aircraft_Damage_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Libraries

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

pd.set_option('display.max_columns', None) # To represent all the columns in the dataset

# Loading Data

In [12]:
Aviation = pd.read_csv('/content/drive/MyDrive/My ML Projects/AviationData.csv', encoding='mac_roman')

In [13]:
Aviation.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,Injury.Severity,Aircraft.damage,Aircraft.Category,Registration.Number,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,NC6404,Stinson,108-3,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N5069P,Piper,PA24-180,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),Destroyed,NaN,N5142R,Cessna,172M,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,N1168J,Rockwell,112,No,1.0,Reciprocating,NaN,NaN,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),Destroyed,NaN,N15NY,Cessna,501,No,NaN,NaN,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [14]:
Aviation.shape

(87282, 31)

There are 87282 flights have been captured to make a prediction

# Data Description

Non-Null Values, Datatypes, % of null values

In [18]:
info_dict = {
    "Column": Aviation.columns.tolist(),
    "Non-Null Count" : Aviation.count(),
    "Dtypes": Aviation.dtypes.tolist(),
    "Null Precentage":(Aviation.shape[0]-Aviation.count())/Aviation.shape[0]*100
}

Description = pd.DataFrame(info_dict)


In [19]:
Description

,Column,Non-Null Count,Dtypes,Null Precentage
Event.Id,Event.Id,87282,object,0.000000
Investigation.Type,Investigation.Type,87282,object,0.000000
Accident.Number,Accident.Number,87282,object,0.000000
Event.Date,Event.Date,87282,object,0.000000
Location,Location,87230,object,0.059577
Country,Country,87056,object,0.258931
Latitude,Latitude,33073,object,62.107880
Longitude,Longitude,33064,object,62.118192
Airport.Code,Airport.Code,49217,object,43.611512
Airport.Name,Airport.Name,51870,object,40.571939
